In [71]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf


In [72]:
#Search directory for all jpgs and returns a list
mylist = os.listdir("./1000subset/MorphII_cropped")
#Converts list into numpy array
mylist = np.array(mylist)
##Builds a an array of all zeros
lab = np.zeros((len(mylist),), dtype=np.int)
lab = np.resize(lab,(1000,2))

In [73]:
#Searchs strings in mylist for M (male) and returns value of 1 if exists and 0 for female if doesn't exist
for i in range(len(mylist)):
    if "M" in mylist[i]:
        lab[i,0] = 1
        lab[i,1] = 0
    else:
        lab[i,0] = 0
        lab[i,1] = 1


In [74]:
#Find list of all filenames in directory
import glob
filelist = glob.glob('./1000subset/MorphII_cropped/*.JPG')
#Convert images into numpy arrays
train = np.array([np.array(Image.open(fname)) for fname in filelist])
#Transforms the 3d array into a 2d array (1000,4200)
train = train.reshape((train.shape[0],-1))
#buils batchs of training and label data


train_batch = tf.train.batch([train], batch_size=100)
label_batch = tf.train.batch([lab], batch_size=100)

In [75]:
#Build a function to convert training data and labels into mini-batches of user-defined size
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)



Xtr, Ytr = next_batch(100, train, lab)
print(Ytr.shape)

(100, 2)


In [83]:
import tensorflow as tf

n_inputs = 4200
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 2

In [84]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [85]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [86]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [87]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

In [88]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=leaky_relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=leaky_relu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

In [89]:
with tf.name_scope("loss"):
    xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [90]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [100]:
with tf.name_scope("eval"):
    correct = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

ValueError: Tensor("ArgMax_2:0", shape=(1000,), dtype=int64) must be from the same graph as Tensor("eval_6/ArgMax:0", dtype=int64).

In [92]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [93]:
n_epochs = 40
batch_size = 50

In [94]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(len(train) // batch_size):
            sess.run(training_op, feed_dict={X: Xtr, y: Ytr})
        if epoch % 5 == 0:
            acc_train = accuracy.eval(feed_dict={X: train, y: lab})
            acc_test = accuracy.eval(feed_dict={X: train, y: lab})
            print(epoch, "Batch accuracy:", acc_train, "Validation accuracy:", acc_test)

    save_path = saver.save(sess, "./my_model_final.ckpt")

InvalidArgumentError: targets must be 1-dimensional
	 [[Node: eval/InTopK = InTopK[T=DT_INT64, k=1, _device="/job:localhost/replica:0/task:0/cpu:0"](dnn_1/outputs/BiasAdd, _recv_y_0)]]

Caused by op 'eval/InTopK', defined at:
  File "/Users/d3r3zz3d/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/d3r3zz3d/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/d3r3zz3d/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/d3r3zz3d/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/d3r3zz3d/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/d3r3zz3d/anaconda/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self)